<a href="https://colab.research.google.com/github/guumaster/colab-nbs/blob/master/fastai/exercises/pytorch_mnist_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook gallery-dl
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *


     |████████████████████████████████| 719 kB 21.4 MB/s 
     |████████████████████████████████| 548 kB 45.1 MB/s 
     |████████████████████████████████| 5.5 MB 44.8 MB/s 
     |████████████████████████████████| 441 kB 8.0 MB/s 
     |████████████████████████████████| 1.3 MB 16.1 MB/s 
     |████████████████████████████████| 1.6 MB 46.7 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 43.6 MB/s 
     |████████████████████████████████| 163 kB 71.1 MB/s 
     |████████████████████████████████| 95 kB 1.6 MB/s 
     |████████████████████████████████| 127 kB 72.0 MB/s 
     |████████████████████████████████| 115 kB 57.9 MB/s 
     |████████████████████████████████| 7.6 MB 62.4 MB/s 
Mounted at /content/gdrive


In [2]:
!pip install -U gallery-dl gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 28.4 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 106 kB 73.9 MB/s 
     |████████████████████████████████| 84 kB 4.7 MB/s 
     |████████████████████████████████| 84 kB 4.4 MB/s 
     |████████████████████████████████| 2.3 MB 51.0 MB/s 
     |████████████████████████████████| 272 kB 52.2 MB/s 
     |████████████████████████████████| 213 kB 55.2 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 80 kB 5.4 MB/s 
     |████████████████████████████████| 68 kB 4.6 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 4.1 MB 45.1 MB/s 
     |████████████████████████████████| 856 kB 5

In [3]:
!python3 -m gallery_dl  -q https://www.pinterest.es/guumaster/dragons/ \
   --write-info-json  --write-metadata --sleep 0.3-0.5 --range 1-210

!python3 -m gallery_dl -q https://www.pinterest.es/guumaster/monsters/ \
   --write-info-json  --write-metadata --sleep 0.3-0.7 --range 1-210 


In [4]:
!mkdir -p train-images/{dragon,monster}/
!mv /content/gallery-dl/pinterest/guumaster/Dragons/* train-images/dragon/
!mv /content/gallery-dl/pinterest/guumaster/Monsters/* train-images/monster/


In [5]:
path="/content/train-images"
fns = get_image_files(path)
fns

(#418) [Path('/content/train-images/dragon/pinterest_182395853646361834.jpg'),Path('/content/train-images/dragon/pinterest_182395853646418367.jpg'),Path('/content/train-images/dragon/pinterest_182395853646361838.png'),Path('/content/train-images/dragon/pinterest_182395853646363441.jpg'),Path('/content/train-images/dragon/pinterest_182395853646417578.jpg'),Path('/content/train-images/dragon/pinterest_182395853646417583.jpg'),Path('/content/train-images/dragon/pinterest_182395853646384366.jpg'),Path('/content/train-images/dragon/pinterest_182395853646398180.jpg'),Path('/content/train-images/dragon/pinterest_182395853646361698.jpg'),Path('/content/train-images/dragon/pinterest_182395853646361840.jpg')...]

In [10]:
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

train = datasets.MNIST(root="sample_data", download=True, train=True, transform=ToTensor())

dataset = DataLoader(train, 32, shuffle=True)



In [8]:
class ImageClassifier(nn.Module):
  def __init__(self):
      super(ImageClassifier, self).__init__()
      self.model = nn.Sequential(
          nn.Conv2d(1, 32, (3,3)),
          nn.ReLU(),
          nn.Conv2d(32, 64, (3,3)),
          nn.ReLU(),
          nn.Conv2d(64, 64, (3,3)),
          nn.ReLU(),
          nn.Flatten(),
          # NOTE: 28-6 related to removed pixels on previous layers. Why?
          nn.Linear(64*(28-6)*(28-6), 10)
      )

  def forward(self, x):
    return self.model(x)


In [9]:
clf = ImageClassifier().to('cuda')

opt = Adam(clf.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(10):
  for batch in dataset:
    X, y = batch
    X, y = X.to('cuda'), y.to('cuda')
    yhat = clf(X)
    loss = loss_fn(yhat, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

  print(f"Epoch {epoch} loss is {loss.item()}")

with open('model.pt', 'wb') as f:
  save(clf.state_dict(), f)



In [20]:
import requests 

img_url = "https://raw.githubusercontent.com/nicknochnack/PyTorchin15/main/img_3.jpg"

img = Image.open(requests.get(img_url, stream = True).raw)

img

In [21]:
import torch
from PIL import Image

with open('model.pt', 'rb') as f:
  clf.load_state_dict(load(f))
  img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

  print(torch.argmax(clf(img_tensor)))
  

tensor(9, device='cuda:0')


In [22]:
ToTensor??